# Hourly temperature averages in Qatar

#### Load python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

## Get World Cup matches by date/time

In [5]:
sked_df = pd.read_csv(
    "data/processed/world_cup_matches_schedule.csv",
    parse_dates=["local_date", "pst_date"],
)

In [6]:
sked_df["time"] = pd.to_datetime(sked_df["local_time"]).dt.strftime("%H:%M")
sked_df["date"] = pd.to_datetime(sked_df["local_date"]).dt.strftime("%m-%d")

## Get weather averages from [weatherspark](https://weatherspark.com/s/105083/2/Average-Fall-Weather-in-Doha-Qatar#Figures-Temperature)

In [7]:
src = pd.read_csv("data/raw/Daily_Averages_for_149641_Hamad_International_Airport.csv")

In [8]:
src.columns = (
    src.columns.str.lower()
    .str.strip()
    .str.replace("/", "_", regex=False)
    .str.replace(" (°f)", "", regex=False)
    .str.replace(" (%)", "_pct", regex=False)
    .str.replace(" (in)", "_inch", regex=False)
    .str.replace(" (hr)", "_hour", regex=False)
    .str.replace(" (mph)", "_mph", regex=False)
    .str.strip()
)

In [9]:
src["time"] = pd.to_datetime(src["timestamp"]).dt.strftime("%H:%M")
src["date"] = pd.to_datetime(src["timestamp"]).dt.strftime("%m-%d")

#### Just the temperature columns

In [10]:
temp_cols = [col for col in src.columns if "temperature_" in col]
pct_cols = [col for col in src.columns if "_pct" in col]

In [13]:
src.head()

,timestamp,temperature_high_mean,temperature_high_percentile10th,temperature_high_percentile25th,temperature_high_percentile75th,temperature_high_percentile90th,temperature_low_mean,temperature_low_percentile10th,temperature_low_percentile25th,temperature_low_percentile75th,temperature_low_percentile90th,temperature_mean,perceivedtemperature_high,perceivedtemperature_low,cloudcover_clear_pct,cloudcover_mostlyclear_pct,cloudcover_partlycloudy_pct,cloudcover_mostlycloudy_pct,cloudcover_overcast_pct,precipitation_total_pct,precipitation_rain_pct,precipitation_mixed_pct,precipitation_snow_pct,precipitation_quantity_mean_inch,rainfall_mean_inch,rainfall_percentile10th_inch,rainfall_percentile25th_inch,rainfall_percentile75th_inch,rainfall_percentile90th_inch,snowfall_mean_inch,snowfall_percentile10th_inch,snowfall_percentile25th_inch,snowfall_percentile75th_inch,snowfall_percentile90th_inch,snowfall_liquidequivalent_mean_inch,snowfall_liquidequivalent_percentile10th_inch,snowfall_liquidequivalent_percentile25th_inch,snowfall_liquidequivalent_percentile75th_inch,snowfall_liquidequivalent_percentile90th_inch,sun_2022_previoussolarmidnight_hour,sun_2022_astronomicalrise_hour,sun_2022_nauticalrise_hour,sun_2022_civilrise_hour,sun_2022_rise_hour,sun_2022_noon_hour,sun_2022_set_hour,sun_2022_civilset_hour,sun_2022_nauticalset_hour,sun_2022_astronomicalset_hour,sun_2022_nextsolarmidnight_hour,daylight_2022_day_hour,daylight_2022_civilday_hour,daylight_2022_nauticalday_hour,daylight_2022_astronomicalday_hour,humidity_dry_pct,humidity_comfortable_pct,humidity_humid_pct,humidity_muggy_pct,humidity_oppressive_pct,humidity_miserable_pct,wind_speed_mean_mph,wind_speed_percentile10th_mph,wind_speed_percentile25th_mph,wind_speed_percentile75th_mph,wind_speed_percentile90th_mph,wind_direction_cardinal_calm_pct,wind_direction_cardinal_n_pct,wind_direction_cardinal_e_pct,wind_direction_cardinal_s_pct,wind_direction_cardinal_w_pct,wind_direction_ordinal_calm_pct,wind_direction_ordinal_n_pct,wind_direction_ordinal_ne_pct,wind_direction_ordinal_e_pct,wind_direction_ordinal_se_pct,wind_direction_ordinal_s_pct,wind_direction_ordinal_sw_pct,wind_direction_ordinal_w_pct,wind_direction_ordinal_nw_pct,wind_direction_display_calm_pct,wind_direction_display_n8_pct,wind_direction_display_ne8partofn4_pct,wind_direction_display_ne8partofe4_pct,wind_direction_display_e8_pct,wind_direction_display_se8partofe4_pct,wind_direction_display_se8partofs4_pct,wind_direction_display_s8_pct,wind_direction_display_sw8partofs4_pct,wind_direction_display_sw8partofw4_pct,wind_direction_display_w8_pct,wind_direction_display_nw8partofw4_pct,wind_direction_display_nw8partofn4_pct,watertemperature_mean,watertemperature_percentile10th,watertemperature_percentile25th,watertemperature_percentile75th,watertemperature_percentile90th,besttime_temperaturetourismscore (),besttime_temperaturebeachscore (),besttime_cloudscore (),besttime_precipitationscore (),besttime_tourismscore (),besttime_beachscore (),growingseasonprobability_pct,temperature_frigid_pct,temperature_freezing_pct,temperature_chilly_pct,temperature_cold_pct,temperature_cool_pct,temperature_comfortable_pct,temperature_warm_pct,temperature_hot_pct,temperature_sweltering_pct,growingdegreedays_mean,growingdegreedays_percentile10th,growingdegreedays_percentile25th,growingdegreedays_percentile75th,growingdegreedays_percentile90th,solarenergy_mean (kwh),solarenergy_percentile10th (kwh),solarenergy_percentile25th (kwh),solarenergy_percentile75th (kwh),solarenergy_percentile90th (kwh),time,date
0,2022-01-01,72.13,66.06,69.07,75.12,78.32,59.15,53.75,56.22,61.84,64.82,65.35,71.68,58.94,58.84,9.14,7.32,6.71,17.97,3.31,3.31,0.0,0.0,0.28,0.28,0.0,0.0,0.35,0.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.38,4.96,5.43,5.90,6.32,11.62,16.92,17.34,17.81,18.28,23.62,10.60,11.43,12.38,13.32,51.06,27.36,16.59,4.80,0.17,0.0,10.33,4.21,6.60,13.73,17.03,0.0,56.47,11.30,17.11,15.13,0.0,19.08,4.51,4.77,10.79,8.61,3.74,3.76,44.75,0.0,19.08,2.45,2.06,4.

In [11]:
src_slim = src[
    [
        "date",
        "time",
        "temperature_mean",
        "temperature_percentile10th",
        "temperature_percentile25th",
        "temperature_percentile75th",
        "temperature_percentile90th",
        "temperature_frigid_pct",
        "temperature_freezing_pct",
        "temperature_chilly_pct",
        "temperature_cold_pct",
        "temperature_cool_pct",
        "temperature_comfortable_pct",
        "temperature_warm_pct",
        "temperature_hot_pct",
        "temperature_sweltering_pct",
    ]
].copy()

KeyError: "['temperature_percentile10th', 'temperature_percentile25th', 'temperature_percentile75th', 'temperature_percentile90th'] not in index"

In [ ]:
src_melt = pd.melt(
    src_slim,
    id_vars=["date", "time"],
    value_vars=[
        "temperature_mean",
        "temperature_percentile10th",
        "temperature_percentile25th",
        "temperature_percentile75th",
        "temperature_percentile90th",
        "temperature_frigid_pct",
        "temperature_freezing_pct",
        "temperature_chilly_pct",
        "temperature_cold_pct",
        "temperature_cool_pct",
        "temperature_comfortable_pct",
        "temperature_warm_pct",
        "temperature_hot_pct",
        "temperature_sweltering_pct",
    ],
    var_name="category",
    value_name="value",
)

In [ ]:
category_list = [
    "temperature_frigid_pct",
    "temperature_freezing_pct",
    "temperature_chilly_pct",
    "temperature_cold_pct",
    "temperature_cool_pct",
    "temperature_comfortable_pct",
    "temperature_warm_pct",
    "temperature_hot_pct",
    "temperature_sweltering_pct",
]

In [ ]:
category_df = src_melt[src_melt["category"] == "temperature_mean"]

In [ ]:
len(category_df)

In [ ]:
category_df

In [ ]:
sked_df.head(1)

In [ ]:
matches = (
    alt.Chart(sked_df).mark_circle(size=3, color="black").encode(x="date:O", y="time:O")
)
matches.properties(width=800)

In [ ]:
category_df.head()

In [ ]:
heatmap = (
    alt.Chart(category_df)
    .mark_rect()
    .encode(
        x=alt.X(
            "date:O",
            axis=alt.Axis(
                values=["01-01", "04-01", "07-01", "10-01", "12-01"], title=" "
            ),
        ),
        y=alt.Y(
            "time:O",
            axis=alt.Axis(
                values=["00:00", "06:00", "12:00", "18:00", "23:00"], title=" "
            ),
        ),
        color=alt.Color(
            "value:Q",
            title="Avg. hourly temp",
            scale=alt.Scale(scheme="spectral", reverse=True),
        ),
    )
)
heatmap.properties(width=800)

In [ ]:
dates_df = pd.DataFrame(
    [{"x_min": "11-21", "x_max": "12-18", "y_min": "00:00", "y_max": "23:00"}]
)

In [ ]:
dates = (
    alt.Chart(dates_df)
    .mark_rect(color="#e9e9e9", opacity=0.3)
    .encode(
        x=alt.X("x_min:O", title=" "),
        x2="x_max:O",
        y=alt.Y("y_min", title=" "),
        y2="y_max",
    )
)

In [ ]:
(
    heatmap.properties(width=400, height=500)
    + matches.properties(width=400, height=500)
    + dates.properties(width=400, height=500)
).configure_legend(orient="top")